# Assignment 2: Building Linear Regression Model using all the 3 methods: Normal Equation, Gradient Descent and Stochistic Gradient Descent

## Importing all important libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
import math

## Reading the given "insurance.txt" file

In [4]:
data = pd.read_csv('insurance.txt', header=0, delimiter=",",quoting=3, encoding='latin-1')
print(data)
age = list(data['age'])
bmi = list(data['bmi'])
children = list(data['children'])
charges = list(data['charges'])
values = []
for i in range(len(age)):
    values.append([age[i], bmi[i], children[i], charges[i]])
      

      age     bmi  children      charges
0      19  27.900         0  16884.92400
1      18  33.770         1   1725.55230
2      28  33.000         3   4449.46200
3      33  22.705         0  21984.47061
4      32  28.880         0   3866.85520
...   ...     ...       ...          ...
1333   50  30.970         3  10600.54830
1334   18  31.920         0   2205.98080
1335   18  36.850         0   1629.83350
1336   21  25.800         0   2007.94500
1337   61  29.070         0  29141.36030

[1338 rows x 4 columns]


## Standardization  z = (xi – μ)/σ.

In [3]:
def feature_scaling(lines):
    for i in range(4):
        tmp = []
        for line in lines:
            tmp.append(line[i])
        tmp = np.array(tmp)
        tmp_mean = tmp.mean()
        tmp_std = tmp.std(ddof = 0)
        for line in lines:            
            line[i] = (line[i] - tmp_mean) / tmp_std
            
    return lines

## Function to Random Shuffle the values

In [4]:
def random_shuffle(values,ratio):
    random.shuffle(values)
    limit=int(math.ceil(len(values) * ratio))
    train = values[: limit]
    test = values[limit: ]
    x_train, y_train, x_test, y_test = [], [], [], []
    for i in train:
        x_train.append(i[: 3])
        y_train.append(i[3])
    for i in test:
        x_test.append(i[: 3])
        y_test.append(i[3])
    x_train = np.array([np.array(i) for i in x_train])
    ones=np.ones((x_train.shape[0],1))
    x_train = np.append(ones, x_train, axis=1)
    y_train = np.array([np.array(i) for i in y_train])
    x_test = np.array([np.array(i) for i in x_test])
    ones=np.ones((x_test.shape[0],1))
    x_test = np.append(ones, x_test, axis=1)
    y_test = np.array([np.array(i) for i in y_test])
    return x_train, y_train, x_test, y_test

In [5]:
values = feature_scaling(values)
rmse_array_nq=[]  # Testing
rmse_array_gd=[] # Testing
rmse_array_sgd=[]  # Testing

rmse_array_nq_train=[] # Training
rmse_array_gd_train=[] # Training
rmse_array_sgd_train=[] # Training

## Solving Using Normal Equations Method

In [6]:
def NormalEquations(X,y):
    A =[]
    B=[]
    C=np.dot(X.T,X)
    D=np.dot(np.linalg.inv(C),np.dot(X.T,y))
    return D

## Solving using Gradient Descent Method

In [7]:
def gradient_descent(X, y,alpha):
    print(type(X))
    cost_array=[]
    theta=np.array([1,1,1,1])
    #alpha = 0.00001     
    i=1
    max_itr=10000
    prev_cost=0
    m = y.size
    while 1:        
        cost, error = cost_function(X, y, theta)
        if(i>=max_itr):
            break
        if(i%50==0):
            print("Iteration",i,end=' :')
            print("Cost is",cost)
        i=i+1                
        theta = theta - (alpha * (1) * np.dot(X.T, error))
        cost_array.append(cost)
        if(abs(prev_cost-cost)<=1e-6):
            break
        prev_cost=cost
    return theta,cost_array

## Solving Using Stochistic Gradient Descent Method

In [8]:
def sgd(epochs,X,Y,learning_rate): #Method for Stochastic Gradient Descent
    #learning_rate = 0.0005
    lr=learning_rate
    m = Y.size
    wt=np.array([1,1,1,1])
    cost_array_sgd=[]
    for itr in range(epochs):
        dvt_wt = np.zeros(wt.shape) 
        error = (1/(2*m))*np.transpose((X@wt - Y))@(X@wt - Y)
        
        if(itr%50==0):
            print("Iteration",itr,end=' :')
            print("Cost is",error)
            cost_array_sgd.append(error)
        # if(abs(error - prev_error) < 0.000001): break
        # prev_error = error
        #error_list.append(error)
        i = random.randint(0,len(X)-1)
        # common = Y[i] - (wt[0] + wt[1]*X[i][1] + wt[2]*X[i][2] + wt[3]*X[i][3])
        common = Y[i] - X[i]@wt
        dvt_wt = -common*X[i]
        wt = wt - lr*dvt_wt
    return wt,cost_array_sgd

## Function that calculates the Root Mean Square Error

In [9]:
def TestingModel(theta,x_test,y_test):
    y_pred=np.dot(x_test,theta)    
    result = sum(np.square(y_pred-y_test))
    rmse = np.sqrt(result/len(x_test))
    return rmse

## Function that calculates Mean and Variance

In [10]:
def Mean_and_Variance(arrayList):
    mean=np.mean(arrayList)
    variance=np.var(arrayList)
    return mean,variance

## Cost Function of Gradient Descent

In [11]:
def cost_function(X, y, theta):
    m = y.size
    error = np.dot(X, theta.T) - y
    cost = 1/(2*m) * np.dot(error.T, error)
    return cost, error

## Building 20 regression models using 1. Gradient Descent Method. 2. Stochistic Gradient Descent Method. 3. Normal Equation Method
#### A new Model is made every time and for each model, the above mentioned 3 algorithms are run. The error calculated after every 50 iterations are shown.

In [12]:
for i in range(20):
    x_train, y_train, x_test, y_test = random_shuffle(values,0.7)
    print("***MODEL NUMBER***",i+1)
    print("Using Gradient Descent")    
    theta_gd,cost_num  = gradient_descent(x_train, y_train,0.00001)
    
    print("Using Stochastic Gradient Method")
    theta_sgd,cost_num_sgd = sgd(100000,x_train,y_train,0.0005)
    print("Weights using Gradient Descent are:",theta_gd)
    print("Weights using Stochistic Gradient Method are",theta_sgd)
    normal_equation=NormalEquations(x_train,y_train)
    print("Weights Using Normal Equations Method are:",normal_equation)
                
    rmse_gd=TestingModel(theta_gd,x_test,y_test) # Testing Data
    rmse_nq=TestingModel(normal_equation,x_test,y_test) # Testing Data
    rmse_sgd=TestingModel(theta_sgd,x_test,y_test) # Testing Data

    rmse_array_gd.append(rmse_gd)
    rmse_array_nq.append(rmse_nq)
    rmse_array_sgd.append(rmse_sgd)
    
    rmse_gd_training=TestingModel(theta_gd,x_train,y_train)
    rmse_nq_training=TestingModel(normal_equation,x_train,y_train)
    rmse_sgd_training=TestingModel(theta_sgd,x_train,y_train)
    
    
    rmse_array_nq_train.append(rmse_nq_training)
    rmse_array_sgd_train.append(rmse_sgd_training)
    rmse_array_gd_train.append(rmse_gd_training)
    


***MODEL NUMBER*** 1
Using Gradient Descent
<class 'numpy.ndarray'>
Iteration 50 :Cost is 1.0552263333429137
Iteration 100 :Cost is 0.6648303905569463
Iteration 150 :Cost is 0.5222387551718307
Iteration 200 :Cost is 0.4698615331191025
Iteration 250 :Cost is 0.45050815668340516
Iteration 300 :Cost is 0.4433129744745317
Iteration 350 :Cost is 0.4406208041168194
Iteration 400 :Cost is 0.43960678673460957
Iteration 450 :Cost is 0.4392222129093091
Iteration 500 :Cost is 0.4390753139271299
Using Stochastic Gradient Method
Iteration 0 :Cost is 2.0959972119658996
Iteration 50 :Cost is 2.0064381610742337
Iteration 100 :Cost is 1.899054906182723
Iteration 150 :Cost is 1.817127864445225
Iteration 200 :Cost is 1.7195225431074506
Iteration 250 :Cost is 1.6388599227155667
Iteration 300 :Cost is 1.561678094062179
Iteration 350 :Cost is 1.508026492096635
Iteration 400 :Cost is 1.448938140934136
Iteration 450 :Cost is 1.3883753433684207
Iteration 500 :Cost is 1.3516905422794907
Iteration 550 :Cost is 1

Iteration 10700 :Cost is 0.43943900526621715
Iteration 10750 :Cost is 0.43946382311546295
Iteration 10800 :Cost is 0.4395110657926543
Iteration 10850 :Cost is 0.43963471676302424
Iteration 10900 :Cost is 0.4396294148436913
Iteration 10950 :Cost is 0.4398071190055825
Iteration 11000 :Cost is 0.43982129767818245
Iteration 11050 :Cost is 0.4398671626183215
Iteration 11100 :Cost is 0.4400056839417045
Iteration 11150 :Cost is 0.4399890306257338
Iteration 11200 :Cost is 0.44000845217661677
Iteration 11250 :Cost is 0.43969849786312537
Iteration 11300 :Cost is 0.43977731094045025
Iteration 11350 :Cost is 0.4397087026311143
Iteration 11400 :Cost is 0.439671438363937
Iteration 11450 :Cost is 0.43964017652547727
Iteration 11500 :Cost is 0.43975511479750795
Iteration 11550 :Cost is 0.43972876920818826
Iteration 11600 :Cost is 0.43968670578087343
Iteration 11650 :Cost is 0.4394242800568799
Iteration 11700 :Cost is 0.4394645455712495
Iteration 11750 :Cost is 0.4394615282321866
Iteration 11800 :Cost 

Iteration 22600 :Cost is 0.43953135806388943
Iteration 22650 :Cost is 0.4394718534181341
Iteration 22700 :Cost is 0.439386411222875
Iteration 22750 :Cost is 0.43938704782621585
Iteration 22800 :Cost is 0.43922410501819675
Iteration 22850 :Cost is 0.4392392245546384
Iteration 22900 :Cost is 0.4393545284430229
Iteration 22950 :Cost is 0.4394175948201505
Iteration 23000 :Cost is 0.43942781562942257
Iteration 23050 :Cost is 0.43967391406534917
Iteration 23100 :Cost is 0.4395743929373778
Iteration 23150 :Cost is 0.43942711821537633
Iteration 23200 :Cost is 0.4394821768701553
Iteration 23250 :Cost is 0.4394429824445879
Iteration 23300 :Cost is 0.4393192657742101
Iteration 23350 :Cost is 0.4391980269973365
Iteration 23400 :Cost is 0.43912288591917054
Iteration 23450 :Cost is 0.43911552176739244
Iteration 23500 :Cost is 0.439087622065062
Iteration 23550 :Cost is 0.4391057669876403
Iteration 23600 :Cost is 0.43904165900839176
Iteration 23650 :Cost is 0.43908964472751627
Iteration 23700 :Cost is

Iteration 33750 :Cost is 0.43943368357039075
Iteration 33800 :Cost is 0.43958575060328664
Iteration 33850 :Cost is 0.43960879561653876
Iteration 33900 :Cost is 0.43955270166172633
Iteration 33950 :Cost is 0.4397425516549721
Iteration 34000 :Cost is 0.4394634207061207
Iteration 34050 :Cost is 0.4396175295877945
Iteration 34100 :Cost is 0.4396379923546143
Iteration 34150 :Cost is 0.4396276292937289
Iteration 34200 :Cost is 0.43938278277636145
Iteration 34250 :Cost is 0.43931503391513793
Iteration 34300 :Cost is 0.4392028040544016
Iteration 34350 :Cost is 0.4392767072500385
Iteration 34400 :Cost is 0.4392611537269873
Iteration 34450 :Cost is 0.4393397892883828
Iteration 34500 :Cost is 0.43935467082559654
Iteration 34550 :Cost is 0.43935669554650186
Iteration 34600 :Cost is 0.4393962654117331
Iteration 34650 :Cost is 0.4392793001861353
Iteration 34700 :Cost is 0.43928143639145123
Iteration 34750 :Cost is 0.4394242523760416
Iteration 34800 :Cost is 0.4393572235360844
Iteration 34850 :Cost i

Iteration 45950 :Cost is 0.4397243409814308
Iteration 46000 :Cost is 0.4397738264919312
Iteration 46050 :Cost is 0.4396983259336319
Iteration 46100 :Cost is 0.43953759898408706
Iteration 46150 :Cost is 0.4395434479074898
Iteration 46200 :Cost is 0.43940173728323384
Iteration 46250 :Cost is 0.43961974630947054
Iteration 46300 :Cost is 0.4397181295655763
Iteration 46350 :Cost is 0.4397914858178429
Iteration 46400 :Cost is 0.43967453017589964
Iteration 46450 :Cost is 0.4398386634472814
Iteration 46500 :Cost is 0.43980021190264096
Iteration 46550 :Cost is 0.4399252850940834
Iteration 46600 :Cost is 0.4398596676933785
Iteration 46650 :Cost is 0.4397615606266498
Iteration 46700 :Cost is 0.439757838036976
Iteration 46750 :Cost is 0.4399095291443972
Iteration 46800 :Cost is 0.439800210829962
Iteration 46850 :Cost is 0.4397386853186712
Iteration 46900 :Cost is 0.43993220988857945
Iteration 46950 :Cost is 0.4399335156183438
Iteration 47000 :Cost is 0.43984935129501945
Iteration 47050 :Cost is 0.

...

Iteration 22800 :Cost is 0.4489468848775652
Iteration 22850 :Cost is 0.4489993342371396
Iteration 22900 :Cost is 0.44898830158557446
Iteration 22950 :Cost is 0.44896027669841493
Iteration 23000 :Cost is 0.44905976756568616
Iteration 23050 :Cost is 0.4490609348600174
Iteration 23100 :Cost is 0.44893411486704665
Iteration 23150 :Cost is 0.44893341430222977
Iteration 23200 :Cost is 0.44898528987646197
Iteration 23250 :Cost is 0.44901944688184614
Iteration 23300 :Cost is 0.4489274193618797
Iteration 23350 :Cost is 0.4489233940243467
Iteration 23400 :Cost is 0.44885294312161217
Iteration 23450 :Cost is 0.44890904885347216
Iteration 23500 :Cost is 0.4490617807540075
Iteration 23550 :Cost is 0.4490234657499072
Iteration 23600 :Cost is 0.44910280917467243
Iteration 23650 :Cost is 0.44899910488883565
Iteration 23700 :Cost is 0.44894365398443914
Iteration 23750 :Cost is 0.44886567787698595
Iteration 23800 :Cost is 0.44878344175797547
Iteration 23850 :Cost is 0.4487908494611673
Iteration 23900 :C

Iteration 34900 :Cost is 0.44902239701996727
Iteration 34950 :Cost is 0.44922188342008185
Iteration 35000 :Cost is 0.4492743936038308
Iteration 35050 :Cost is 0.44961030303865035
Iteration 35100 :Cost is 0.4496728642751052
Iteration 35150 :Cost is 0.44953480525734335
Iteration 35200 :Cost is 0.44947033526762714
Iteration 35250 :Cost is 0.4492432326494746
Iteration 35300 :Cost is 0.4493959459357707
Iteration 35350 :Cost is 0.44926792068349564
Iteration 35400 :Cost is 0.44917855426789255
Iteration 35450 :Cost is 0.4491458368315097
Iteration 35500 :Cost is 0.4491753552715908
Iteration 35550 :Cost is 0.4491249759191406
Iteration 35600 :Cost is 0.44901186486376365
Iteration 35650 :Cost is 0.4492561069627562
Iteration 35700 :Cost is 0.4490414800179702
Iteration 35750 :Cost is 0.4490514736667084
Iteration 35800 :Cost is 0.4490668983211226
Iteration 35850 :Cost is 0.44907908922687784
Iteration 35900 :Cost is 0.4490667891064781
Iteration 35950 :Cost is 0.4489563303124249
Iteration 36000 :Cost i

Iteration 47850 :Cost is 0.44887464992514164
Iteration 47900 :Cost is 0.4489206923211167
Iteration 47950 :Cost is 0.4488299426546117
Iteration 48000 :Cost is 0.44890398317927616
Iteration 48050 :Cost is 0.44890118347658936
Iteration 48100 :Cost is 0.4489474354933991
Iteration 48150 :Cost is 0.44915379784349063
Iteration 48200 :Cost is 0.449223892938016
Iteration 48250 :Cost is 0.44922119136341593
Iteration 48300 :Cost is 0.4491271392600808
Iteration 48350 :Cost is 0.4490799594935784
Iteration 48400 :Cost is 0.4490269755604931
Iteration 48450 :Cost is 0.4489373570298116
Iteration 48500 :Cost is 0.44887500981573214
Iteration 48550 :Cost is 0.44883750218688334
Iteration 48600 :Cost is 0.4488129241048966
Iteration 48650 :Cost is 0.4489014425085914
Iteration 48700 :Cost is 0.44888128219790757
Iteration 48750 :Cost is 0.4488648385672597
Iteration 48800 :Cost is 0.44893601581459325
Iteration 48850 :Cost is 0.4489456555427993
Iteration 48900 :Cost is 0.44885577124297354
Iteration 48950 :Cost i

Iteration 60550 :Cost is 0.4490144796803932
Iteration 60600 :Cost is 0.4490441704097845
Iteration 60650 :Cost is 0.44916050847080813
Iteration 60700 :Cost is 0.449037426042608
Iteration 60750 :Cost is 0.4489936630211381
Iteration 60800 :Cost is 0.44902858077507785
Iteration 60850 :Cost is 0.44910767618324676
Iteration 60900 :Cost is 0.4490150457457145
Iteration 60950 :Cost is 0.4489927273817742
Iteration 61000 :Cost is 0.44892487879541887
Iteration 61050 :Cost is 0.44898193723418
Iteration 61100 :Cost is 0.4490204538263875
Iteration 61150 :Cost is 0.44921898243756386
Iteration 61200 :Cost is 0.4495234639881447
Iteration 61250 :Cost is 0.44957002593275625
Iteration 61300 :Cost is 0.44962447848852016
Iteration 61350 :Cost is 0.44950477692465024
Iteration 61400 :Cost is 0.4494018786398935
Iteration 61450 :Cost is 0.44930436218382674
Iteration 61500 :Cost is 0.4493233450053614
Iteration 61550 :Cost is 0.4494994872437834
Iteration 61600 :Cost is 0.4494054833262361
Iteration 61650 :Cost is 0

KeyboardInterrupt: 

### Printing Error calculated for normal Equation Methods for each of the 20 different models

In [ ]:
print("For Normal Equations")    
for i in range(20):
    print("MODEL NUMBER",i+1,end='')
    print("  RMSE error using normal Equation-:",rmse_array_nq[i])
mean_nq,variance_nq=Mean_and_Variance(rmse_array_nq)
print("Mean RMSE error for Normal Equation Method=",mean_nq)
print("Variance for Normal Equation method=",variance_nq)

In [ ]:
print("Comparing Accuracy of each model for all 3 methods:")    
for i in range(20):
    print("MODEL NUMBER",i+1)
    print("  RMSE error using normal Equation-:",rmse_array_nq[i])
    print("  RMSE error using Gradient Descent-:",rmse_array_gd[i])
    print("  RMSE error using Stochastic Gradient Descent-:",rmse_array_sgd[i])
    print()


In [ ]:
mean_gd,variance_gd=Mean_and_Variance(rmse_array_gd)
mean_nq,variance_nq=Mean_and_Variance(rmse_array_nq)
mean_sgd,variance_sgd=Mean_and_Variance(rmse_array_sgd)
print("Mean error of Stochastic gradient method is",mean_sgd)
print("Variance error of Stochastic gradient method is",variance_sgd)
print("Minimum RMSE error for Stochastic Gradient for testing",min(rmse_array_sgd))
print("Minimum RMSE error for Stochastic Gradient for training",min(rmse_array_sgd_train))
print()    
print("Mean error of Normal Equation method is",mean_nq)
print("Variance error of Normal Equation method is",variance_nq)
print("Minimum RMSE error for Normal Equation for testing",min(rmse_array_nq))
print("Minimum RMSE error for Normal Equation for training",min(rmse_array_nq_train))
print()    
print("Mean error of Gradient Descent method is",mean_gd)
print("Variance error of Gradient Descent method is",variance_gd)
print("Minimum RMSE error for Gradient Descent for testing",min(rmse_array_gd))
print("Minimum RMSE error for Gradient Descent for training",min(rmse_array_gd_train))

## Plotting The Error Vs Epochs Graph using Gradient Descent

In [ ]:
def plotChart(iterations, cost_num):
    a=[1,2,3,4]
    print(type(a))
    #cost_num=np.array(cost_num)
    #print(type(cost_num))
    ite=[]
    for j in range(len(cost_num)):
        ite.append(j+1)
    #iterations=np.array(ite)        
    #print(type(iterations))
    iterations=ite
    fig, ax = plt.subplots()
    ax.plot(iterations, cost_num,linewidth=1.0)
    ax.set_xlabel('Iterations')
    ax.set_ylabel('Cost')
    ax.set_title('Error vs Iterations')
    plt.style.use('fivethirtyeight')
    plt.show()

In [ ]:
plotChart(100000,cost_num)

## Plotting The Error Vs Epochs Graph using Stochistic Gradient Descent

In [ ]:
plotChart(10000,cost_num_sgd)

## Visualizing the Error Vs Epochs for Different Learning Rates for Gradient Descent and Stochistic Gradient Descent

### For that 5 new Models are again made with different Learning rates for Gradient Descent and Stochistic Gradient Descent

In [ ]:
Learning_rates_array=[0.00001, 0.000005, 0.000001]
cost_num_gd_LR=[]
for i in range(3):
    print(i)
    x_train, y_train, x_test, y_test = random_shuffle(values,0.7)
    theta_gd,cost_num  = gradient_descent(x_train, y_train,Learning_rates_array[i])
    cost_num_gd_LR.append(cost_num)
    

In [ ]:
min_x=1000
Learning_rates_array=[0.00001, 0.000005, 0.000001]
plt.figure(figsize=(10,10))
#print(len(cost_num_gd_LR[0]))
for i in range(len(cost_num_gd_LR)):
    if((len(cost_num_gd_LR[i]))<min_x):
        min_x=len(cost_num_gd_LR[i])

for i in range(len(cost_num_gd_LR)):
    x_axis=[]  
    y_axis=[]
    for j in range(min_x):
        x_axis.append(j)
        y_axis.append(cost_num_gd_LR[i][j])
        
 
    plt.plot(x_axis,y_axis,linewidth=1.0)
legend=[]
for i in range(len(cost_num_gd_LR)):
    legend.append(f'Alpha = {Learning_rates_array[i]}')
    
plt.xlabel("Cost")
plt.ylabel("Iterations")
plt.legend(legend)   
#print(min_x)

## Visualizing the Error Vs Epochs for Different Learning Rates for Stochastic Gradient Descent 

### For that 5 new Models are again made with different Learning rates for Stochistic Gradient Descent

In [ ]:
Learning_rates_array=[0.001, 0.0005, 0.00001]
cost_num_sgd_LR=[]
for i in range(3):
    print(i)
    x_train, y_train, x_test, y_test = random_shuffle(values,0.7)
    theta_sgd,cost_num_sgd_LR_returned  = sgd(100000,x_train,y_train,Learning_rates_array[i])
    cost_num_sgd_LR.append(cost_num_sgd_LR_returned)

In [ ]:
min_x=1000
Learning_rates_array=[0.001, 0.0005, 0.00001]
plt.figure(figsize=(10,10))
#print(len(cost_num_gd_LR[0]))
for i in range(len(cost_num_sgd_LR)):
    if((len(cost_num_sgd_LR[i]))<min_x):
        min_x=len(cost_num_sgd_LR[i])

for i in range(len(cost_num_sgd_LR)):
    x_axis=[]  
    y_axis=[]
    for j in range(min_x):
        x_axis.append(j)
        y_axis.append(cost_num_sgd_LR[i][j])
        
 
    plt.plot(x_axis,y_axis,linewidth=1.0)
legend=[]
for i in range(len(cost_num_sgd_LR)):
    legend.append(f'Alpha = {Learning_rates_array[i]}')
    
plt.xlabel("Cost")
plt.ylabel("Iterations")
plt.legend(legend)   
#print(min_x)